In [5]:
import kgbench as kg

data = kg.load('mdgenre') # Load with numpy arrays, and train/validation split

loaded data mdgenre (83.45s).


In [6]:
print(data.num_classes)
print(data.datatypes())

12
['iri', 'http://kgbench.info/dt#base64Image', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#langString', 'http://www.w3.org/2001/XMLSchema#dateTime', 'http://www.w3.org/2001/XMLSchema#decimal', 'http://www.w3.org/2001/XMLSchema#string']


In [7]:
data.withheld[:15]

array([[263997,      1],
       [139883,      9],
       [220756,     11],
       [233568,      2],
       [141778,      5],
       [218163,      2],
       [147763,      4],
       [149772,     11],
       [155895,      2],
       [184777,      2],
       [118631,      2],
       [218571,      6],
       [255150,      2],
       [219682,     11],
       [155134,     11]], dtype=int32)

In [8]:
def create_uri_classes_lists(data, data_set):
    list_of_uris = []
    list_of_classes=[]
    for data_instance in data_set:
    #     print(data_instance)
        list_of_uris.append(data.i2e[data_instance[0]][0])
        list_of_classes.append(data_instance[1])
    return list_of_uris, list_of_classes

In [9]:
list_of_uris_train, list_of_classes_train = create_uri_classes_lists(data, data.training)

In [10]:
list_of_uris, list_of_classes = create_uri_classes_lists(data, data.withheld)

In [11]:
def show_lists(list1, list2):
    for i, (uri, y) in enumerate(zip(list1, list2)):
        print(uri, y)
        if i > 1:
            break
    print(len(list1), len(list2))

print("witheld:")
show_lists(list_of_uris, list_of_classes)
print("train:")
show_lists(list_of_uris_train, list_of_classes_train)


witheld:
http://www.wikidata.org/entity/Q86081603 1
http://www.wikidata.org/entity/Q1894071 9
http://www.wikidata.org/entity/Q5352076 11
500 500
train:
http://www.wikidata.org/entity/Q602283 2
http://www.wikidata.org/entity/Q987943 2
http://www.wikidata.org/entity/Q18198101 11
4863 4863


In [12]:
data.i2e[127365]

('http://www.wikidata.org/entity/Q16424132', 'iri')

In [13]:
import time

In [15]:
import rdflib
start_time = time.time()
g = rdflib.Graph()
g.parse("data/md_raw_without_Images-.nt")
print("--- %s seconds ---" % (time.time() - start_time))

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x7f7f5c842670>
Traceback (most recent call last):
  File "/Users/roderickvanderweerdt/opt/anaconda3/envs/gcn_env/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "/Users/roderickvanderweerdt/opt/anaconda3/envs/gcn_env/lib/python3.9/site-packages/isodate/isodates.py", line 203, in parse_date
    raise ISO8601Error('Unrecognised ISO 8601 date format: %r' % datestring)
isodate.isoerror.ISO8601Error: Unrecognised ISO 8601 date format: '-0800-01-01T00:00:00Z'
Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x7f7f5c842670>
Traceback (most recent call last):
  File "/Users/roderickvanderweerdt/opt/anaconda3/envs/gcn_env/lib/python3.9/site-packages/rdflib/term.py", line 2084, in _castLexica

--- 56.10393786430359 seconds ---


In [19]:
q_duration = """
SELECT DISTINCT ?movie_uri ?val
WHERE {
    ?movie_uri <http://www.wikidata.org/prop/direct/P2047> ?val .
}
"""

q_props = """
SELECT DISTINCT ?z
WHERE {
  ?x a ?z .
}
"""

def run_query(graph, query):
    qres = graph.query(query)
    for row in qres:
        print(row)
        break
        
run_query(g, q_duration)

(rdflib.term.URIRef('http://www.wikidata.org/entity/Q2549028'), rdflib.term.Literal('20', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#decimal')))


In [16]:
q = """
SELECT DISTINCT ?entity_uri ?val
WHERE {{
    ?entity_uri {} ?val .
}}
"""


#monument_values = {year: <int>, population: <int>, codeNationalMonument: <int>}
#movie_values = {duration: <decimal>, boxoffice: <decimal>}
def add_entity_value(entity_val_dict, value_name, graph, query):
    qres = graph.query(query)
    print("query returned", len(qres), "results.")
    new_dict = 0
    existing_dict = 0
    doubles = 0
    for row in qres:
        entity_str = str(row.entity_uri)
        try:
            try:
#                 monument_val_dict[monument_str][value_name] = int((int(monument_val_dict[monument_str][value_name]) + int(row.val))/2)
                if entity_val_dict[entity_str][value_name] > float(row.val):
                    entity_val_dict[entity_str][value_name] = float(row.val)
#                 print(entity_val_dict[entity_str][value_name], row.val)
                doubles += 1
            except:
                entity_val_dict[entity_str][value_name] = float(row.val)
                existing_dict += 1
        except:
            new_dict += 1
            entity_val_dict[entity_str] = {}
            entity_val_dict[entity_str][value_name] = float(row.val)
    print("values for new entity:", new_dict, "\nvalues for previous entities:", existing_dict, "\nmultiple values for same entities:", doubles)
    print("entity_val_dict now contains values for", len(entity_val_dict), "entities.\n")
    return entity_val_dict

value_list = ["duration", "boxoffice", "cost"]
props_list = ["<http://www.wikidata.org/prop/direct/P2047>", "<http://www.wikidata.org/prop/direct/P2142>", "<http://www.wikidata.org/prop/direct/P2130>"]
entity_val_dict = {}
start_time = time.time()
for value_name, prop in zip(value_list, props_list):
    entity_val_dict = add_entity_value(entity_val_dict, value_name, g, q.format(prop))
print("--- %s seconds ---" % (time.time() - start_time))

query returned 8163 results.
values for new entity: 8099 
values for previous entities: 0 
multiple values for same entities: 64
entity_val_dict now contains values for 8099 entities.

query returned 844 results.
values for new entity: 2 
values for previous entities: 787 
multiple values for same entities: 55
entity_val_dict now contains values for 8101 entities.

query returned 698 results.
values for new entity: 0 
values for previous entities: 696 
multiple values for same entities: 2
entity_val_dict now contains values for 8101 entities.

--- 0.6214659214019775 seconds ---


In [19]:
def print_values(val_list, val_dict):
    values = ""
    for val in val_list:
        try:
            values += str(val_dict[val])+','
        except:
            values += ','
    return values[:-1] #remove last ','
        
def write_entities_md(list_of_entities, list_of_y, val_list, entity_val_dict, new_fn):
    un_found = [0,0]
    with open(new_fn, 'w') as f:
        f.write("entity,y,"+','.join(val_list)+'\n')
        for entity, y in zip(list_of_entities, list_of_y):
            try:
                f.write(entity+','+str(y)+','+print_values(val_list, entity_val_dict[entity])+'\n')
                un_found[0] += 1
            except:
                f.write(entity+','+str(y)+','+print_values(val_list, {})+'\n')
#                 print(entity+','+str(y)+','+print_values(val_list, {}))
#                 print("no values found for entity:", entity)
                un_found[1] += 1
    print("found values for", un_found[0], "entities")
    print("couldn't find values for", un_found[1], "entities")

In [28]:
write_entities_md(list_of_uris_train, 
                   list_of_classes_train, 
                   value_list, 
                   entity_val_dict, 
                   "data/entities_md_raw_TRAIN.csv")

write_entities_md(list_of_uris, 
                   list_of_classes, 
                   value_list, 
                   entity_val_dict,
                   "data/entities_md_raw_VALIDATION.csv")


found values for 4236 entities
couldn't find values for 627 entities
found values for 426 entities
couldn't find values for 74 entities


In [20]:
shuffle_dataset("data/entities_md_raw_TRAIN.csv")
shuffle_dataset("data/entities_md_raw_VALIDATION.csv")

In [1]:
import pandas as pd

In [29]:
def combine_files(update_fn, train_fn, combined_fn):
    update_entities_df = pd.read_csv(update_fn, sep=",")
    train_entities_df = pd.read_csv(train_fn, sep=",")
    
    combined_entities_df = pd.concat([train_entities_df, update_entities_df])
    combined_entities_df.to_csv(combined_fn, index=False)
    
def subtract_files(complete_fn, subtract_fn, result_fn, entity_column="entity"):
    complete_df = pd.read_csv(complete_fn)
    subtract_df = pd.read_csv(subtract_fn)
    complete_df.drop(complete_df[complete_df[entity_column].isin(subtract_df[entity_column])].index, inplace = True)
    complete_df.to_csv(result_fn, index=False)

In [30]:
update_filename = "data/entities_md_raw_VALIDATION.csv"
train_filename = "data/entities_md_raw_TRAIN.csv"
combined_filename = "data/entities_md_raw_COMBINED.csv"

combine_files(update_filename, train_filename, combined_filename)

CREATE EMBEDDINGS

In [1]:
emb_file = "data/entities_md_raw_COMBINED_w_emb.csv"

update_w_emb_fn = "data/entities_md_raw_VALIDATION_w_emb.csv"
train_w_emb_fn = "data/entities_md_raw_TRAIN_w_emb.csv"

# subtract_files(emb_file, train_filename, update_w_emb_fn)
# subtract_files(emb_file, update_filename, train_w_emb_fn)

In [31]:
emb_file = "data/md_raw_without_Images_w_emb.csv"

update_fn = "data/entities_md_raw_VALIDATION.csv"
train_fn = "data/entities_md_raw_TRAIN.csv"
update_w_emb_fn = "data/entities_md_raw_VALIDATION_w_emb.csv"
train_w_emb_fn = "data/entities_md_raw_TRAIN_w_emb.csv"

subtract_files(emb_file, train_fn, update_w_emb_fn)
subtract_files(emb_file, update_fn, train_w_emb_fn)

In [3]:
from classifier_mlp_AM import perform_prediction


In [4]:
final_results = perform_prediction(dataset_fn=train_w_emb_fn, show_all=True, save_model=True)

Shape of X [N, C, H, W]:  torch.Size([8, 100])
Shape of y:  torch.Size([8]) torch.int64
Using cpu device
creating a classifier for 12 classes.
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=100, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=12, bias=True)
  )
)
Epoch 1
-------------------------------
loss: 2.526779  [    0/ 4863]
loss: 2.044061  [  800/ 4863]
loss: 1.996471  [ 1600/ 4863]
loss: 1.710418  [ 2400/ 4863]
loss: 1.537046  [ 3200/ 4863]
loss: 2.606719  [ 4000/ 4863]
loss: 1.290527  [ 4800/ 4863]
Train Accuracy: 54.6%
Test Accuracy: 60.8%
Epoch 2
-------------------------------
loss: 1.077845  [    0/ 4863]
loss: 1.656432  [  800/ 4863]
loss: 1.867676  [ 1600/ 4863]
loss: 1.351381  [ 2400/ 4863]
loss: 1.385845  [ 3200/ 4863]
loss: 2.346861  [ 4000/ 4863]
loss: 1.194376  [ 4800/ 4

loss: 1.153948  [ 2400/ 4863]
loss: 0.948260  [ 3200/ 4863]
loss: 0.900477  [ 4000/ 4863]
loss: 0.695054  [ 4800/ 4863]
Train Accuracy: 69.3%
Test Accuracy: 65.0%
Epoch 28
-------------------------------
loss: 0.647576  [    0/ 4863]
loss: 0.971575  [  800/ 4863]
loss: 1.196845  [ 1600/ 4863]
loss: 1.150604  [ 2400/ 4863]
loss: 0.938207  [ 3200/ 4863]
loss: 0.890163  [ 4000/ 4863]
loss: 0.683671  [ 4800/ 4863]
Train Accuracy: 69.4%
Test Accuracy: 65.0%
Epoch 29
-------------------------------
loss: 0.636681  [    0/ 4863]
loss: 0.964869  [  800/ 4863]
loss: 1.187857  [ 1600/ 4863]
loss: 1.146528  [ 2400/ 4863]
loss: 0.931098  [ 3200/ 4863]
loss: 0.879549  [ 4000/ 4863]
loss: 0.671607  [ 4800/ 4863]
Train Accuracy: 69.8%
Test Accuracy: 64.8%
Epoch 30
-------------------------------
loss: 0.630195  [    0/ 4863]
loss: 0.958910  [  800/ 4863]
loss: 1.178417  [ 1600/ 4863]
loss: 1.142497  [ 2400/ 4863]
loss: 0.922006  [ 3200/ 4863]
loss: 0.871985  [ 4000/ 4863]
loss: 0.660287  [ 4800/ 4863